In [2]:
# ============================================
# CELL 1: Import Libraries
# ============================================
import pandas as pd
import numpy as np
import os
import re

print("="*60)
print("🧹 FINAL DATA CLEANING FOR ATHENA")
print("="*60)

🧹 FINAL DATA CLEANING FOR ATHENA


In [3]:
# ============================================
# CELL 2: Load Original Data
# ============================================

# Load from Downloads folder (original clean data)
df = pd.read_csv(r"C:\Users\ACER\Downloads\netflix_customer_churn.csv")

print(f"✅ Loaded: {len(df)} rows, {len(df.columns)} columns")
print("\n👀 First 3 rows:")
print(df.head(3))

✅ Loaded: 5000 rows, 14 columns

👀 First 3 rows:
                            customer_id  age  gender subscription_type  \
0  a9b75100-82a8-427a-a208-72f24052884a   51   Other             Basic   
1  49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b   47   Other          Standard   
2  4d71f6ce-fca9-4ff7-8afa-197ac24de14b   27  Female          Standard   

   watch_hours  last_login_days  region  device  monthly_fee  churned  \
0        14.73               29  Africa      TV         8.99        1   
1         0.70               19  Europe  Mobile        13.99        1   
2        16.32               10    Asia      TV        13.99        0   

  payment_method  number_of_profiles  avg_watch_time_per_day favorite_genre  
0      Gift Card                   1                    0.49         Action  
1      Gift Card                   5                    0.03         Sci-Fi  
2         Crypto                   2                    1.48          Drama  


In [4]:
# ============================================
# CELL 3: Remove Garbage Rows
# ============================================

# Check for any rows with garbage data
print("🔍 Checking for garbage rows...")

# Function to check if customer_id is valid UUID format
def is_valid_uuid(val):
    pattern = r'^[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}$'
    return bool(re.match(pattern, str(val), re.IGNORECASE))

# Filter valid rows
df['is_valid'] = df['customer_id'].apply(is_valid_uuid)
invalid_rows = df[~df['is_valid']].shape[0]

print(f"Invalid rows found: {invalid_rows}")

# Keep only valid rows
df = df[df['is_valid']].copy()
df = df.drop('is_valid', axis=1)

print(f"✅ After cleaning: {len(df)} rows")

🔍 Checking for garbage rows...
Invalid rows found: 0
✅ After cleaning: 5000 rows


In [5]:
# ============================================
# CELL 4: Add Numeric Customer ID
# ============================================

# Keep original UUID as separate column
df['customer_uuid'] = df['customer_id']

# Add numeric ID (1 to N)
df['customer_id'] = range(1, len(df) + 1)

# Reorder columns
cols = ['customer_id', 'customer_uuid'] + [col for col in df.columns if col not in ['customer_id', 'customer_uuid']]
df = df[cols]

print("✅ Numeric customer_id added")
print("\n👀 First 3 rows:")
print(df[['customer_id', 'customer_uuid', 'age', 'gender']].head(3))

✅ Numeric customer_id added

👀 First 3 rows:
   customer_id                         customer_uuid  age  gender
0            1  a9b75100-82a8-427a-a208-72f24052884a   51   Other
1            2  49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b   47   Other
2            3  4d71f6ce-fca9-4ff7-8afa-197ac24de14b   27  Female


In [6]:
# ============================================
# CELL 5: Fix Data Types
# ============================================

# Age - ensure integer
df['age'] = pd.to_numeric(df['age'], errors='coerce').fillna(30).astype(int)

# Watch hours - ensure float
df['watch_hours'] = pd.to_numeric(df['watch_hours'], errors='coerce').fillna(0).round(2)

# Last login days - ensure integer
df['last_login_days'] = pd.to_numeric(df['last_login_days'], errors='coerce').fillna(0).astype(int)

# Monthly fee - ensure float
df['monthly_fee'] = pd.to_numeric(df['monthly_fee'], errors='coerce').fillna(9.99).round(2)

# Churned - ensure integer (0/1)
df['churned'] = pd.to_numeric(df['churned'], errors='coerce').fillna(0).astype(int)
df['churned'] = df['churned'].apply(lambda x: 1 if x > 0 else 0)

# Number of profiles - ensure integer
df['number_of_profiles'] = pd.to_numeric(df['number_of_profiles'], errors='coerce').fillna(1).astype(int)
df['number_of_profiles'] = df['number_of_profiles'].clip(1, 10)

# Avg watch time - ensure float
df['avg_watch_time_per_day'] = pd.to_numeric(df['avg_watch_time_per_day'], errors='coerce').fillna(0).round(2)

print("✅ Data types fixed")
print("\n📊 Data types:")
print(df.dtypes)

✅ Data types fixed

📊 Data types:
customer_id                 int64
customer_uuid              object
age                         int32
gender                     object
subscription_type          object
watch_hours               float64
last_login_days             int32
region                     object
device                     object
monthly_fee               float64
churned                     int64
payment_method             object
number_of_profiles          int32
avg_watch_time_per_day    float64
favorite_genre             object
dtype: object


In [7]:
# ============================================
# CELL 6: Clean Categorical Columns
# ============================================

# Gender
df['gender'] = df['gender'].fillna('Unknown')
df['gender'] = df['gender'].replace({
    'M': 'Male', 'F': 'Female', 'm': 'Male', 'f': 'Female',
    'male': 'Male', 'female': 'Female', 'other': 'Other', 'Other': 'Other'
})

# Subscription type
df['subscription_type'] = df['subscription_type'].fillna('Basic')
df['subscription_type'] = df['subscription_type'].replace({
    'basic': 'Basic', 'BASIC': 'Basic',
    'standard': 'Standard', 'STANDARD': 'Standard',
    'premium': 'Premium', 'PREMIUM': 'Premium'
})

# Region
df['region'] = df['region'].fillna('Unknown')
valid_regions = ['Asia', 'Africa', 'Europe', 'North America', 'South America', 'Oceania']
df['region'] = df['region'].apply(lambda x: x if x in valid_regions else 'Unknown')

# Device
df['device'] = df['device'].fillna('Unknown')
valid_devices = ['Mobile', 'TV', 'Laptop', 'Desktop', 'Tablet']
df['device'] = df['device'].apply(lambda x: x if x in valid_devices else 'Unknown')

# Payment method
df['payment_method'] = df['payment_method'].fillna('Unknown')
valid_payments = ['Credit Card', 'Debit Card', 'PayPal', 'Gift Card', 'Crypto']
df['payment_method'] = df['payment_method'].apply(lambda x: x if x in valid_payments else 'Unknown')

# Favorite genre
df['favorite_genre'] = df['favorite_genre'].fillna('Unknown')
valid_genres = ['Action', 'Comedy', 'Drama', 'Horror', 'Sci-Fi', 'Romance', 'Documentary']
df['favorite_genre'] = df['favorite_genre'].apply(lambda x: x if x in valid_genres else 'Other')

print("✅ Categorical columns cleaned")
print("\n📊 Value counts:")
for col in ['gender', 'subscription_type', 'region', 'device', 'payment_method']:
    print(f"\n{col}:")
    print(df[col].value_counts().head())

✅ Categorical columns cleaned

📊 Value counts:

gender:
gender
Female    1711
Male      1654
Other     1635
Name: count, dtype: int64

subscription_type:
subscription_type
Premium     1693
Basic       1661
Standard    1646
Name: count, dtype: int64

region:
region
South America    873
Europe           867
North America    851
Asia             841
Africa           803
Name: count, dtype: int64

device:
device
Tablet     1048
Laptop     1006
Mobile     1004
TV          993
Desktop     949
Name: count, dtype: int64

payment_method:
payment_method
Debit Card     1030
PayPal         1026
Crypto          995
Gift Card       976
Credit Card     973
Name: count, dtype: int64


In [8]:
# ============================================
# CELL 7: Add Business Metrics
# ============================================

# Annual revenue
df['annual_revenue'] = (df['monthly_fee'] * 12).round(2)

# Customer Lifetime Value (3 years)
df['clv'] = (df['annual_revenue'] * 3).round(2)

# Risk Score (0-100)
max_login = df['last_login_days'].max()
max_watch = df['watch_hours'].max()
max_avg = df['avg_watch_time_per_day'].max()

login_risk = (df['last_login_days'] / max_login * 40).clip(0, 40)
watch_risk = ((max_watch - df['watch_hours']) / max_watch * 40).clip(0, 40)
avg_risk = ((max_avg - df['avg_watch_time_per_day']) / max_avg * 20).clip(0, 20)

df['risk_score'] = (login_risk + watch_risk + avg_risk).round(1)

# Risk categories
df['risk_category'] = 'LOW'
df.loc[df['risk_score'] >= 60, 'risk_category'] = 'MEDIUM'
df.loc[df['risk_score'] >= 75, 'risk_category'] = 'HIGH'
df.loc[df['risk_score'] >= 90, 'risk_category'] = 'CRITICAL'

# Revenue at risk
df['revenue_at_risk'] = (df['risk_score'] / 100 * df['annual_revenue']).round(2)

print("✅ Business metrics added")
print(f"\n📊 Risk distribution:")
print(df['risk_category'].value_counts())

✅ Business metrics added

📊 Risk distribution:
risk_category
HIGH        1857
MEDIUM      1769
CRITICAL     774
LOW          600
Name: count, dtype: int64


In [9]:
# ============================================
# CELL 8: Remove customer_id = 1 (Optional)
# ============================================

print(f"Before removal: {len(df)} rows")

# Check if customer_id = 1 exists
if 1 in df['customer_id'].values:
    print(f"✅ customer_id = 1 exists")
    
    # Remove it
    df = df[df['customer_id'] != 1].copy()
    
    # Reset customer_id to be sequential
    df['customer_id'] = range(1, len(df) + 1)
    
    print(f"After removal: {len(df)} rows")
    print(f"✅ customer_id = 1 removed and IDs re-sequenced")
else:
    print(f"customer_id = 1 not found")

Before removal: 5000 rows
✅ customer_id = 1 exists
After removal: 4999 rows
✅ customer_id = 1 removed and IDs re-sequenced


In [10]:
print("="*50)
print("DATA QUALITY CHECK")
print("="*50)

print("\n1. Total rows:", len(df))
print("2. Total columns:", len(df.columns))
print("3. Missing values:", df.isnull().sum().sum())
print("4. Duplicate IDs:", df['customer_id'].duplicated().sum())
print("5. Customer IDs:", df['customer_id'].min(), "to", df['customer_id'].max())

print("\n6. Sample data:")
display(df.head(3))  # Jupyter Notebook mein display() use karo

print("\n7. Churn Rate:", round(df['churned'].mean()*100, 1), "%")

DATA QUALITY CHECK

1. Total rows: 4999
2. Total columns: 20
3. Missing values: 0
4. Duplicate IDs: 0
5. Customer IDs: 1 to 4999

6. Sample data:


,customer_id,customer_uuid,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre,annual_revenue,clv,risk_score,risk_category,revenue_at_risk
1,1,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,0.03,Sci-Fi,167.88,503.64,72.4,MEDIUM,121.55
2,2,4d71f6ce-fca9-4ff7-8afa-197ac24de14b,27,Female,Standard,16.32,10,Asia,TV,13.99,0,Crypto,2,1.48,Drama,167.88,503.64,60.5,MEDIUM,101.57
3,3,d3c72c38-631b-4f9e-8a0e-de103cad1a7d,53,Other,Premium,4.51,12,Oceania,TV,17.99,1,Crypto,2,0.35,Horror,215.88,647.64,66.3,MEDIUM,143.13



7. Churn Rate: 50.3 %


In [11]:
# ============================================
# CELL 10: Save Cleaned Data
# ============================================

# Create output folder
output_folder = r"C:\Users\ACER\Desktop\OTT_Churn_Project\data\athena_ready"
os.makedirs(output_folder, exist_ok=True)

# Save CSV (without index)
output_file = os.path.join(output_folder, "churn_athena_ready.csv")
df.to_csv(output_file, index=False)

print(f"✅ Saved: {output_file}")
print(f"   File size: {os.path.getsize(output_file)/1024:.1f} KB")
print(f"   Rows: {len(df)}")
print(f"   Columns: {len(df.columns)}")

# Show first 2 rows for verification
print("\n📋 First 2 rows (for Athena):")
print(df.head(2).to_string(index=False))

✅ Saved: C:\Users\ACER\Desktop\OTT_Churn_Project\data\athena_ready\churn_athena_ready.csv
   File size: 716.4 KB
   Rows: 4999
   Columns: 20

📋 First 2 rows (for Athena):
 customer_id                        customer_uuid  age gender subscription_type  watch_hours  last_login_days region device  monthly_fee  churned payment_method  number_of_profiles  avg_watch_time_per_day favorite_genre  annual_revenue    clv  risk_score risk_category  revenue_at_risk
           1 49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b   47  Other          Standard         0.70               19 Europe Mobile        13.99        1      Gift Card                   5                    0.03         Sci-Fi          167.88 503.64        72.4        MEDIUM           121.55
           2 4d71f6ce-fca9-4ff7-8afa-197ac24de14b   27 Female          Standard        16.32               10   Asia     TV        13.99        0         Crypto                   2                    1.48          Drama          167.88 503.64        60.5  